# Lab 3

### Team Members:
 - Alex Lopez
 - Chris Haub
 - Erin McClure-Price
 - Chad Kwong

#### Library Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
#import turicreate as tc
from sklearn.preprocessing import StandardScaler

#### Data Import

In [2]:
%%time

#Loading all data file separately 
vle = pd.read_csv('./anonymiseddata/vle.csv')
assessments = pd.read_csv('./anonymiseddata/assessments.csv')
courses = pd.read_csv('./anonymiseddata/courses.csv')
studentAssessments = pd.read_csv('./anonymiseddata/studentAssessment.csv')
studentInfo = pd.read_csv('./anonymiseddata/studentInfo.csv')
studentRegistration = pd.read_csv('./anonymiseddata/studentRegistration.csv')
studentVle = pd.read_csv('./anonymiseddata/studentVle.csv')

Wall time: 3.75 s


In [3]:
%%time
dfFull = pd.merge(studentVle, studentInfo, on=['code_module', 'code_presentation', 'id_student'])
print(dfFull.shape)
dfFull.head()

(10655280, 15)
Wall time: 4.11 s


,code_module,code_presentation,id_student,id_site,date,sum_click,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,28400,546652,-10,4,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
1,AAA,2013J,28400,546652,-10,1,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,28400,546652,-10,1,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
3,AAA,2013J,28400,546614,-10,11,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
4,AAA,2013J,28400,546714,-10,1,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass


# Business Understanding 1 #
*Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). How will you measure the effectiveness of a good algorithm? Why does your chosen validation method make sense for this specific dataset and the stakeholders needs?*

# Data Understanding 1 #
*Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file. Verify data quality: Are there missing values? Duplicate data? Outliers? Are those mistakes? How do you deal with these problems?*

 - VLE stands for Virtual Learning Environment
 - VLE table records interactions with VLE 
 - 32,593 students
 - 22 courses
 

In [4]:
vle.head()

,id_site,code_module,code_presentation,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,NaN,NaN
1,546712,AAA,2013J,oucontent,NaN,NaN
2,546998,AAA,2013J,resource,NaN,NaN
3,546888,AAA,2013J,url,NaN,NaN
4,547035,AAA,2013J,resource,NaN,NaN


In [5]:
%%time
# Randomly sampling 4,000 records StudentVle Table with Million Rows.
studentSample = studentInfo.sample(n=4000, random_state = 777)
print(studentSample.shape)

# vle
# assessments X
# courses X
# studentAssessments X
# studentInfo X
# studentRegistration X
# studentVleSample X

df = pd.merge(studentSample, studentVle, on=['code_module', 'code_presentation', 'id_student'])
df2 = pd.merge(studentAssessments, assessments, on='id_assessment')
df = pd.merge(df, df2, on=['code_module', 'code_presentation', 'id_student'], how='left')
df = pd.merge(df, courses, on= ['code_module', 'code_presentation'], how='left')
df = pd.merge(df, studentRegistration, on=['code_module', 'code_presentation', 'id_student'], how='left')
df = pd.merge(df, vle, on=['id_site', 'code_module', 'code_presentation'], how='left')
print(df.shape)
df.head()

(4000, 12)
(10744419, 28)
Wall time: 22.9 s


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,...,score,assessment_type,date_y,weight,module_presentation_length,date_registration,date_unregistration,activity_type,week_from,week_to
0,CCC,2014J,134190,F,Scotland,A Level or Equivalent,30-40%,0-35,0,120,...,78.0,CMA,18.0,2.0,269,-70.0,32.0,homepage,NaN,NaN
1,CCC,2014J,134190,F,Scotland,A Level or Equivalent,30-40%,0-35,0,120,...,78.0,CMA,18.0,2.0,269,-70.0,32.0,resource,NaN,NaN
2,CCC,2014J,134190,F,Scotland,A Level or Equivalent,30-40%,0-35,0,120,...,78.0,CMA,18.0,2.0,269,-70.0,32.0,forumng,NaN,NaN
3,CCC,2014J,134190,F,Scotland,A Level or Equivalent,30-40%,0-35,0,120,...,78.0,CMA,18.0,2.0,269,-70.0,32.0,forumng,NaN,NaN
4,CCC,2014J,134190,F,Scotland,A Level or Equivalent,30-40%,0-35,0,120,...,78.0,CMA,18.0,2.0,269,-70.0,32.0,homepage,NaN,NaN


In [6]:
print('Missing values: \n', df.isnull().sum() * 100 / len(df),'\n\n')
df = df.drop(columns = ['week_from', 'week_to', 'date_unregistration'])
print('Missing values after dopping columns: \n', df.isnull().sum() * 100 / len(df))

Missing values: 
 code_module                    0.000000
code_presentation              0.000000
id_student                     0.000000
gender                         0.000000
region                         0.000000
highest_education              0.000000
imd_band                       6.064051
age_band                       0.000000
num_of_prev_attempts           0.000000
studied_credits                0.000000
disability                     0.000000
final_result                   0.000000
id_site                        0.000000
date_x                         0.000000
sum_click                      0.000000
id_assessment                  0.117233
date_submitted                 0.117233
is_banked                      0.117233
score                          0.160958
assessment_type                0.117233
date_y                         1.879394
weight                         0.117233
module_presentation_length     0.000000
date_registration              0.001889
date_unregistration   

In [7]:
df.dtypes

code_module                    object
code_presentation              object
id_student                      int64
gender                         object
region                         object
highest_education              object
imd_band                       object
age_band                       object
num_of_prev_attempts            int64
studied_credits                 int64
disability                     object
final_result                   object
id_site                         int64
date_x                          int64
sum_click                       int64
id_assessment                 float64
date_submitted                float64
is_banked                     float64
score                         float64
assessment_type                object
date_y                        float64
weight                        float64
module_presentation_length      int64
date_registration             float64
activity_type                  object
dtype: object

In [10]:
# View unique values in each column
# for col in df:
#   print(df[col].unique())

In [11]:
#Creating variables for indexing continuous and categorical variables
conCol = ['id_student', 'id_site', 'date_x', 'sum_click', 'num_of_prev_attempts', 
          'studied_credits', 'id_assessment ', 'date_submitted', 'is_banked', 'score', 'date_y', 'weight']

catCol = ['code_module', 'code_presentation', 'gender', 'region', 'highest_education', 
          'imd_band', 'age_band', 'disability', 'final_result', 'assessment_type' ]

In [12]:
#Declaring Scalar object
scl_obj = StandardScaler()

#One hot encoding of ALL categorical variables
OneHotDF = pd.get_dummies(df[catCol],drop_first=False)

#Scaling non-encoded data
conVar = df.select_dtypes(exclude='object')
colnames = pd.Series(conVar.columns)
conVarScaled = scl_obj.fit_transform(conVar)
conVarScaled = pd.DataFrame(data = conVarScaled, columns= colnames)

#Combining with continuous variables from cleaned dataset
OneHotDF = OneHotDF.reset_index()
OneHotDF.pop('index')
OneHotDF = pd.concat([conVarScaled,OneHotDF], axis = 1)
OneHotDF.head()

#https://github.com/jakemdrew/DataMiningNotebooks/blob/master/01.%20Pandas.ipynb


KeyError: "['code_module '] not in index"

# Data Understanding 2 #
*Visualize the any important attributes appropriately. Important: Provide an interpretation for any charts or graphs.*

# Modeling and Evaluation 1 #
*Train and adjust parameters*

# Modeling and Evaluation 2	#
*Evaluate and Compare*

# Modeling and Evaluation 3 #
*Visualize Results*

# Modeling and Evaluation 4 #
*Summarize the Ramifications*

# Deployment #
*Be critical of your performance and tell the reader how you current model might be usable by other parties. Did you achieve your goals? If not, can you reign in the utility of your modeling? How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?*

# Exceptional Work #